# 演習の方針

1. **ベースラインモデル評価**  
   素のモデルで回答を生成し、講義内容との整合性の低さを観察します。これにより、特別な学習なしでのモデルの限界を確認します。

2. **文字起こしデータの活用**  
   講義の文字起こしデータを導入し、モデルが講義内容を参照した回答を生成する傾向を観察します。ただし、Retrieval（情報検索）精度の限界から結果は不安定になる可能性があります。

3. **チャンク化の導入**  
   文字起こしデータをチャンク（小単位）に分割し、より安定して関連コンテンツを取得できるようにします。この段階では文脈理解にまだ課題があることを確認します。

4. **Rerankの適用**  
   検索結果のランク付けを導入し、より的確で安定した回答を目指します。

5. **応用改善手法**  
   文字起こしの品質向上のための編集技術や、メタデータの活用による性能向上手法を探ります。

## 扱う質問

「Inference Time Scaling（推論時スケーリング）」に関する質問を取り扱います。これは以下の背景を持つトピックです。

- 2024年8月発表の論文「Scaling LLM Test-Time Compute Optimally can be More Effective than Scaling Model Parameters」で提唱された概念
- OpenAIのGPT-o1（2024年9月リリース）で実用化され、注目を集めた比較的新しいアプローチ
- 2024年度LLM講座の第4回講義でも取り上げられた重要テーマ

- 模範解答:
```
「Inference Time Scaling」とは、推論時に計算量を増やしてモデルの性能を高める手法です。これはモデルのサイズを大きくする代わりに、難しい入力に対して多くの計算リソースを使うことで、より良い出力を得ようとするアプローチです。
```

## 扱うモデル

Meta-Llama-3-8B-Instruct（2024年4月リリース）を使用します。このモデルは、リリース時期の関係上、以下の特徴を持ちます。

- 「Inference Time Scaling」の概念が広まる前に訓練されており、このトピックに関する知識を持たないと想定される
- この特性を活かし、純粋なベースライン評価から各手法の効果を観察する

### 演習環境の準備

In [1]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

In [2]:
# 演習用のコンテンツを取得
!git clone https://github.com/matsuolab/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52 (from 1)
Receiving objects: 100% (52/52), 83.21 KiB | 20.80 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [3]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import random
random.seed(0)

In [6]:
# モデル(Llama3)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [7]:
def generate_output(query, system_prompt=None):
  if system_prompt is None:
    messages = [
        {"role": "user", "content": query},
    ]
  else:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

- 数値的な評価も見てみます。RagasにはAnswer Accuracyという評価指標があります。今回はこちらを参考に実装した評価関数を利用して測っていきます。

今回はLlama3では性能が不安定だったので、OpenAIのgpt-4oで評価していきます。従って、scoreの実行はopenAI APIキーを所持している関心がある方のみで良いです。

In [8]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。"
question =  "LLMにおけるChinchilla scaling lawsとは？"
response = generate_output(question, system_prompt)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpe

In [9]:
print(response)

LLM（Large Language Model）におけるChinchilla scaling lawsは、Chinchillaという名前のモデルアーキテクチャーに基づくスケーリング法です。Chinchillaは、Googleの研究者によって提案されたLLMのアーキテクチャーで、モデルサイズの増加に対応するために設計されたスケーリング法です。

Chinchilla scaling lawsは、LLMのパラメーターのスケーリングを通じて、モデルサイズの増加に対応するために使用されます。具体的には、Chinchilla scaling lawsは、LLMのパラメーターのスケーリングを、モデルサイズの増加に対応するために使用されます。LLMのパラメーターのスケーリングには、以下のような方法が含まれます。

1. パラメーターのスケーリング：LLMのパラメーターのスケーリングを、モデルサイズの増加に対応するために使用します。
2. モデルのスケーリング：LLMのモデルサイズを、モデルサイズの増加に対応するために使用します。
3. パラメーターの調整


In [10]:
!pip install -U openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.0/662.0 kB 42.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.2
    Uninstalling openai-1.76.2:
      Successfully uninstalled openai-1.76.2


In [ ]:
# # @title 評価実装
# gold_answer = "「Inference Time Scaling」とは、推論時に計算量を増やしてモデルの性能を高める手法です。これはモデルのサイズを大きくする代わりに、難しい入力に対して多くの計算リソースを使うことで、より良い出力を得ようとするアプローチです。"

# from openai import OpenAI
# from google.colab import userdata
# client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"), max_retries=5, timeout=60)

# def openai_generator(query):

#         messages = [
#                     {
#                         "role": "user",
#                         "content": query
#                     }
#                 ]

#         response = client.chat.completions.create(
#             model="gpt-4o-mini",
#             messages=messages
#         )
#         return response.choices[0].message.content

# def evaluate_answer_accuracy(query, response, reference):

#     template_accuracy1 = (
#           "Instruction: You are a world class state of the art assistant for rating "
#           "a User Answer given a Question. The Question is completely answered by the Reference Answer.\n"
#           "Say 4, if User Answer is full contained and equivalent to Reference Answer"
#           "in all terms, topics, numbers, metrics, dates and units.\n"
#           "Say 2, if User Answer is partially contained and almost equivalent to Reference Answer"
#           "in all terms, topics, numbers, metrics, dates and units.\n"
#           "Say 0, if User Answer is not contained in Reference Answer or not accurate in all terms, topics,"
#           "numbers, metrics, dates and units or the User Answer do not answer the question.\n"
#           "Do not explain or justify your rating. Your rating must be only 4, 2 or 0 according to the instructions above.\n"
#           "Even small discrepancies in meaning, terminology, directionality, or implication must result in a lower score. Only rate 4 if the User Answer is a complete and precise match to the Reference Answer in every aspect.\n"
#           "### Question: {query}\n"
#           "### {answer0}: {sentence_inference}\n"
#           "### {answer1}: {sentence_true}\n"
#           "The rating is:\n"
#       )
#     template_accuracy2 = (
#           "I will rate the User Answer in comparison to the Reference Answer for a given Question.\n"
#           "A rating of 4 indicates that the User Answer is entirely consistent with the Reference Answer, covering all aspects, topics, numbers, metrics, dates, and units.\n"
#           "A rating of 2 signifies that the User Answer is mostly aligned with the Reference Answer, with minor discrepancies in some areas.\n"
#           "A rating of 0 means that the User Answer is either inaccurate, incomplete, or unrelated to the Reference Answer, or it fails to address the Question.\n"
#           "I will provide the rating without any explanation or justification, adhering to the following scale: 0 (no match), 2 (partial match), 4 (exact match).\n"
#           "Even minor inconsistencies in meaning, terminology, emphasis, or factual detail should prevent a rating of 4. Only assign a 4 if the User Answer exactly and unambiguously matches the Reference Answer in every respect."
#           "Do not explain or justify my rating. My rating must be only 4, 2 or 0 only.\n\n"
#           "Question: {query}\n\n"
#           "{answer0}: {sentence_inference}\n\n"
#           "{answer1}: {sentence_true}\n\n"
#           "Rating: "
#       )

#     score1 = openai_generator(
#                 template_accuracy1.format(
#                       query=query,
#                       answer0="User Answer",
#                       answer1="Reference Answer",
#                       sentence_inference=response,
#                       sentence_true=reference,
#                     )
#                 )
#     try:
#       score1 = int(score1)
#     except:
#       print("Failed")
#       score1 = 0

#     score2 = openai_generator(
#                 template_accuracy2.format(
#                         query=query,
#                         answer0="Reference Answer",
#                         answer1="User Answer",
#                         sentence_inference=reference,
#                         sentence_true=response,
#                     )
#                   )

#     try:
#       score2 = int(score2)
#     except:
#       print("Failed")
#       score2 = 0


#     return (score1 + score2) / 2

In [ ]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

## 結果 (ベースモデル)

Meta-Llama-3-8B-Instructは「Inference Time Scaling」について誤った知識を提示しました：
* モデルは従来の「推論時間の短縮」という文脈でInference Time Scalingを解釈しており、これはLLM分野における最新の「Inference Time Scaling」概念（推論時計算資源の最適配分）とは異なる説明になります。

---

# 2. 文字起こしデータの活用
## 講義内容をソースとして活用 (RAG導入)

モデルの回答の事実性を向上させるためにRetrieval Augmented Generation (RAG)技術を導入します：

* **知識ソース**: LLM講座第4講における講師の発言内容
* **目的**: モデルに「Inference Time Scaling」に関する正確な知識と文脈を提供し、事実に基づいた回答を促す

**初期RAG実装（ベーシックアプローチ）**:
* **ドキュメント処理**: 音声認識モデル(speech2text)で書き起こした生テキストをそのまま使用
* **分割方法**: 「。」（句点）で区切られた文単位でテキストを分割
* **検索手法**: シンプルな類似度ベースの検索でクエリに関連する文を抽出
* **制約条件**: モデルの入力トークン制限に収まるよう関連文のみを選択

In [11]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 8192

In [12]:
with open("/content/lecture-ai-engineering/day3/data/LLM2024_day4_raw.txt", "r") as f:
  raw_writedown = f.read()

In [13]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[250])

ドキュメントサイズ:  306
ドキュメントの例: 
 このDecodingにもいろんな方法があってグリーンDecodingだと単純に一番いいやつを選んでいく、一番確率が高いやつ選んでいくので、すごい単純ですけど、こういうトップKeyを取るとかトップ系を取るとかして、最後に一番いいやつを選ぶみたいなことをすると、これも結局計算をたくさんしてることになるわけですね


In [14]:
# Retrievalの実行
question = "LLMにおけるChinchilla scaling lawsとは？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[58.466209411621094, 58.867645263671875, 56.2042236328125, 51.5880126953125, 50.16627883911133, 56.065155029296875, 58.61505126953125, 50.535987854003906, 57.962520599365234, 58.72676467895508, 53.0543212890625, 57.022071838378906, 51.332950592041016, 52.45307540893555, 53.87307357788086, 55.608177185058594, 59.28961181640625, 55.40880584716797, 55.60260009765625, 56.814598083496094, 60.8417854309082, 59.53520584106445, 58.844078063964844, 57.796287536621094, 51.61714553833008, 59.911930084228516, 55.77289962768555, 62.166744232177734, 53.554595947265625, 56.303192138671875, 54.79912185668945, 55.213016510009766, 52.73438262939453, 50.01127624511719, 53.539432525634766, 55.44524383544922, 55.469749450683594, 55.499794006347656, 55.84645080566406, 54.591896057128906, 55.33937072753906, 52.01337432861328, 55.783592224121094, 53.64848327636719, 54.39429473876953, 55.47475814819336, 58.490055084228516, 57.57576370239258, 56.04449462890625, 56.62965774536133, 56.70292282104492, 50.29706954

In [15]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 64.78422546386719)
1月に論文としてまして、スケーリングLLMthisTimeコンピュート口真理ちゃん日は増えてるっていうことで、良いらしいというふうに言われてます 


取得したドキュメント2: (Score: 64.00057983398438)
この辺の理屈は小さなモデルだと、学習がロスが下がらなくなるというのでちょっとあのスケール則を書いてるときに、なんか言っこのプロット説明者と同じような話ですけど、あの計算量が与えられたときに、どうやら、別にパラメータを増やせばいいわけではないいうことはわかりこの計算量が与えられたときにどれぐらいのバジェット、どれぐらいのパラメータ数とtoken数に割り振ればいいのかっていうのを、あの計算しようとしたのが、先ほど出したこの2変数の関係っていうふうに言ったChinChillaと呼ばれる経験則になってます 


取得したドキュメント3: (Score: 63.4155158996582)
これがあの係数が20このtokenをパラメータであった値が、20、20倍のtokenを利用しましょうということでしたけど、例えばLlama2だと学び7Billonでも70Billonと同じtokenを使ってますけど、7Billonのものに関しては1.8とBillonのtoken使ってるので、285倍のケース全然違う値を使ってるDense70Billonは28個Llama3に関しても、21470Billonが214倍、400倍Billonの方が37倍ということで、このChinChillaオプティカル実際に巨大なモデル、巨大なtokenの学習を使っているケースもよくあります 


取得したドキュメント4: (Score: 62.86857223510742)
それから1変数を制御するのではなくて複数の変数を制御するような経験則も知られていて有名なBERTChinChillaと呼ばれる経験則がありますChinChilla論文って確かにここまでで計測スケール則の話をしたんですけどChinChillaのところで少しパラメータ数とデータセットサイズ、それぞれいじって計算量を肯定しますよって話をしたのでちょっとその補足をしておきたいと思います 


取得したドキュメント5: (Score: 62.

In [16]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるChinchilla scaling lawsとは？"
response = generate_output(question, system_prompt)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [17]:
print(response)

LLM（Large Language Model）におけるChinchilla scaling lawsは、Chinchillaという経験則に基づいて、パラメータ数、token数、計算量の関係を規定するスケール則です。

Chinchilla scaling lawsは、LLMの学習に必要な計算量を、パラメータ数とtoken数の関係で規定することで、計算量を予測することができます。具体的には、Chinchilla scaling lawsは、パラメータ数とtoken数の関係を、以下の式で規定します。

* パラメータ数：P
* token数：T
* 計算量：C

この式で、計算量Cは、パラメータ数Pとtoken数Tの積に比例することが示されます。すなわち、パラメータ数Pを増やすことで、token数Tを増やすことで、計算量Cを減らすことができます。

Chinchilla scaling lawsは、LLMの学習に必要な計算量を規定することで、計算量を予測することができます。また、Chinchilla scaling lawsは、LLMのパラ


In [ ]:
# # 評価 (openai apikeyがある場合のみ実行)
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

## 結果 (初期RAG実装)

講義内容のドキュメントを追加したにもかかわらず、モデルの回答には依然として以下の問題が見られます：
* 「高速に推論する」など、従来の一般的な推論最適化と「Inference Time Scaling」を混同した誤った解釈が継続
* 講義内容を参照しているものの、概念の本質を正確に捉えられていない

### 問題分析
以下の要因が考えられます：
1. **ドキュメント品質の問題**: 音声認識による文字起こしの精度不足
2. **検索精度の課題**: 単純な文単位の分割では文脈が失われ、関連性の高いドキュメント片を適切に取得できていない可能性

### 書き起こしテキストの品質改善

日本語の音声認識（speech2text）モデルは精度に課題があることが知られています。以下に「LLMにおけるInference Time Scalingとは？」に関連する講義内容の書き起こしテキストを比較します：

### 講義中の該当発言 (LLM講座Day4後半から抜粋)


<修正前>
---

講義に戻ります。ちょっと練習の時間もあるのであと20分ぐらいで駆け足になりますけど、最後最近のスケールトレンドって話で**生のGENIACLM**の話をして終わろうと思いですねちょっとモチベーションから話すと、ちょっと頭で考えてみてほしいとか見れば一瞬で思うとんですけどバナナの色は何ですかって言われたときと、今日の講義聞いた上で、**ゲームソフトの問題は何だと思いますか**って聞かれたとき、多分あの考えることが違うと思うんですね。**羽の色なんですか**っていうと一瞬黄色ですねもしかしたら緑かもしれないけどぐらいですかね物によるかなみたいなおもちゃだったら違うかもみたいな、だんだんあの、考えていくといろいろ出てくるかもしれないすけど、少なくとも**スケール足の問題なんだと思いますか**って聞かれたときに、今日の話からするとスケール則っていうのはこういうものだからどうだろうこの辺が問題かなみたいな考えとやっぱ思考としては違うってことは何となく思うかなと思います。なんか人間的にはこの二つって全然違うしあの、答えるのに必要な考え方っていうのも違うように思えるわけです。**スケールって言ってる7Gのスケール**って言ってるのはこういった形で、あの簡単なものについては簡単に答えてもいいですし、そうじゃなくて、あの考えなきゃいけない問題に対しては、考える時間を、に計算式を使うというふうにしたときに、これいいことがあるのかっていうような話になってます。二つで、ちょっと順番が前後しますけどこれの仕組みは言語モデルでも効果的ですかっていう話と、これをどう実現できるかっていう、こういう二つの話が最近のトレンドとして出てきています。効果的ですかっていうのが、最近**大湾**と呼ばれる論文が論文じゃないか、モデルが**オペル**から出ましたプレビューとして出てますけどこの法案で注目されていますこれあの**論文にROMってかブログ**にあるとイエスって右側が訓練時の計算資源をスケールさせたときに、初めて何かロジックのベンチマークがあるんですけどこれをがどうなったかで何となくスケールしてると右側がテストTimeコンピュートっていうふうに書いてると思うんすけど、**水温時**に計算資源を増やしたときあるモデルを使うんだけど、簡単に答える方法と深く考えて答える方法みたいでだんだんコース計算式を増やしていったときに、性能がどう変わるかっていうのでこれもスケールしていってるということがわかると思います。こういった形で、要は考える時間をどうやら推論時に使うと計算資源を推論使うのはいいことがありそうだということがわかります。


<修正後>
---


講義に戻ります。ちょっと演習の時間もあるのであと20分ぐらいで駆け足になりますけど、最後最近のスケールトレンドってことで**「推論時のスケーリング」**についての話をして終わろうと思います。モチベーションから話すと、ちょっと頭で考えてみてもらえれば一瞬でわかると思うとんですけど、「バナナの色は何ですかって言われたとき」と、今日の講義聞いた上で、**「スケール則の問題は何だと思いますか」**って聞かれたとき、多分あの考えることが違うと思うんですね。
**「バナナの色なんですか」**っていうと黄色ですね。もしかしたら緑かもしれないけど、物によるかなみたいな、おもちゃだったら違うかもみたいな、だんだんあの、考えていくといろいろ出てくるかもしれないすけど、少なくとも**「スケール則の問題なんだと思いますか」**って聞かれたときに、今日の話からするとスケール則っていうのはこういうものだから「どうだろう」「この辺が問題かな」みたいな考えとはやっぱ思考としては違うってことは何となく思うかなと思います。
なんか人間的にはこの二つって全然違うしあの、答えるのに必要な考え方っていうのも違うように思えるわけです。**推論時のスケールって言ってるのは**こういった形で、あの簡単なものについては簡単に答えてもいいですし、そうじゃなくて、深く考えなきゃいけない問題に対しては、考える時間に計算資源を使うというふうにしたときに、これいいことがあるのかっていうような話になってます。
これの仕組みは言語モデルでも効果的ですかっていう話と、これをどう実現できるかっていう、こういう二つの話が最近のトレンドとして出てきています。効果的ですかっていうのが、最近**o1**と呼ばれるモデルが**OpenAI**から出ました。プレビューとして出てますけどこのo1で注目されています。これあのo1の**論文ってかブログ**にある図で、左側が訓練時の計算資源をスケールさせたときに、AIMEというロジックのベンチマークがあるんですけど、accuracyがどうなったかというと、何となくスケールしてる。右側がtest-time computeっていうふうに書いてると思うんすけど、**推論時**に計算資源を増やしたときあるモデルを使うんだけど、簡単に答える方法と深く考えて答える方法みたいでだんだん計算資源を増やしていったときに、性能がどう変わるかっていうので、これもスケールしていってるということがわかると思います。
こういった形で、要は考える時間をどうやら推論時に使うと、つまり計算資源を推論時に使うのはいいことがありそうだということがわかります。






---
### 文字起こしの誤り

上記の比較からわかるように、音声認識による書き起こしには重大な誤りが多数含まれています：
* 「スケール則の問題」→「ゲームソフトの問題」
* 「o1」→「大湾」
といった明らかに文脈に合わない単語変換が発生しています。

`LLM2024_day4_raw.txt`の中には、このような誤変換が多数見られます。これらの誤りはRAG性能に直接影響し、モデルの回答精度を低下させる要因となります。

したがって、**ドキュメント品質の改善**を行い、RAG性能の向上を図ります。

## 講義内容をソースとして活用：改善版RAG実装

* **ドキュメント処理**:
  - speech2textによる書き起こしテキストを人手で丁寧に修正
  - 専門用語（Inference Time Scaling、GPT-o1など）の正確な表記を確保
  - 文脈の流れを維持しつつ、文法的に正確な日本語に修正

* **検索手法**:
  - 引き続き「。」（句点）で区切られた文単位でテキストを分割
  - 文単位の検索により、モデルの入力トークン制限内で関連情報を最大化

この改善により、モデルが正確な情報に基づいて「Inference Time Scaling」の概念を理解し、適切な回答を生成することが期待されます。

In [18]:
with open("/content/lecture-ai-engineering/day3/data/LLM2024_day4.txt", "r") as f:
  raw_writedown = f.read()

In [19]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[310])

ドキュメントサイズ:  350
ドキュメントの例: 
 それからBest of Nとはちょっと違う方法として、N個を生成した後に、それらを集約するという意味では、Day2でやったSelf-Consistencyをこの枠組みの一つとして説明されます


In [20]:
# Retrievalの実行
question = "LLMにおけるChinchilla scaling lawsとは？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[58.231544494628906, 58.551658630371094, 56.95147705078125, 49.53276824951172, 52.455074310302734, 50.16627502441406, 56.065185546875, 58.466102600097656, 51.71729278564453, 53.074241638183594, 57.37200927734375, 58.72675323486328, 53.23201370239258, 57.02206039428711, 51.33295822143555, 52.45307540893555, 53.87309646606445, 49.60065460205078, 56.42440414428711, 59.28964614868164, 55.4088249206543, 57.249534606933594, 56.814605712890625, 60.841796875, 61.37364959716797, 57.1976432800293, 55.24169158935547, 59.148372650146484, 51.61715316772461, 57.661991119384766, 53.171356201171875, 57.68410110473633, 62.166778564453125, 53.21492004394531, 56.30318069458008, 54.628814697265625, 56.567893981933594, 53.73492431640625, 53.53944396972656, 55.44525909423828, 55.92243576049805, 52.791778564453125, 53.48512649536133, 55.95191192626953, 55.62403106689453, 53.648502349853516, 54.69175720214844, 52.57651901245117, 54.51872253417969, 50.154727935791016, 58.50096893310547, 60.316932678222656, 56

In [21]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 64.91248321533203)
ここまでで計測スケール則の話をしたんですけどChinChillaのところで少しパラメータ数とデータセットサイズ、それぞれいじって計算量を固定しますよって話をしたのでちょっとその補足をしておきたいと思います 


取得したドキュメント2: (Score: 63.949031829833984)
この辺の理屈は小さなモデルだと、学習がロスが下がらなくなるというのでちょっとあのスケール則を書いてるときに、なんか1個1個のプロットを説明したと同じような話ですけど、あの計算量が与えられたときに、どうやら、別にパラメータを増やせばいいわけではないいうことはわかりこの計算量が与えられたときにどれぐらいのバジェット、どれぐらいのパラメータ数とtoken数に割り振ればいいのかっていうのを、あの計算しようとしたのが、先ほど出したこの2変数の関係っていうふうに言ったChinChillaと呼ばれる経験則になってます 


取得したドキュメント3: (Score: 63.88928985595703)
Llama3に関しても、70Billonが214倍、400Billonの方が37倍ということで、このChinChilla-Optimalが実際に巨大なモデル、巨大なtokenの学習で使っているケースもよくあります 


取得したドキュメント4: (Score: 62.53028106689453)
あのスケール則についてここまで他の多分一番有名なのが先ほどのから話しているScaling Laws for Neural Language Modelというものなんですけれど、実はこのスケーリングっていうの自体、スケーリング則、スケール則が成立するってこと自体は、もうちょっと前から知られていたというふうに言われています 


取得したドキュメント5: (Score: 62.47397232055664)
ちなみにこれも余談ですけどこのの求め方このChinChilla則ってのは実は何かいくつかの方法で求められてまして、それぞれ大体同じような経験則が出るってことが知られています 




In [22]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるChinchilla scaling lawsとは？"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [23]:
print(response)

LLM（Large Language Model）におけるChinchilla scaling lawsは、ChinChillaという経験則に基づくスケーリング則です。このスケーリング則は、計算量を固定した状態で、パラメータ数とデータセットサイズを変化させることで、LLMの性能を予測するために使用されます。

このスケーリング則は、ChinChillaの経験則に基づいており、計算量を固定した状態で、パラメータ数とデータセットサイズを変化させることで、LLMの性能を予測するために使用されます。このスケーリング則は、LLMの巨大化に伴う計算量の増加を考慮し、パラメータ数とデータセットサイズを適切に設定することで、LLMの性能を向上させることができます。


In [ ]:
# # 評価 (openai apikeyがある場合のみ実行)
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

## 結果 (修正テキストによるRAG)

書き起こしテキストの品質改善により、モデルの回答に部分的な向上が見られました：

### 改善点
* 「推論時（Inference）に計算資源をスケーリングすることで、モデルがより賢くなり、性能が向上すること」という概念を正確に捉えるようになった

### 問題点
* 「Inference Time Scalingは、TransformerやLSTMなどのモデルにおいて、パラメータ数を増やすのではなく、推論時計算資源をスケーリングすることで、性能が向上すること...」という記述は講義内容と矛盾している

### 問題分析

モデルが誤った回答を生成する主要因として、**文脈の欠如**が考えられます：
* 「。」で区切られた短い文単位での検索では、各文の発言背景や関連性が失われる
* 単独の文から情報を抽出するため、講師の全体的な主張や議論の流れを把握できない
* 結果として、正しい個別の文でも、その解釈に必要な背景情報が欠如し、誤った文脈で理解される

# 3. 文脈を考慮したチャンク化の導入

検索結果の品質向上のため、以下の改善を実施します：

* **前後文脈を含むチャンク化**:
  - 検索でマッチした文だけでなく、その前後の複数文も含めてチャンクとして取得
  - 具体的には、マッチした文を中心に前2文、後2文を含む計5文程度のチャンクを構成
  - この「文脈ウィンドウ」により、発言の背景情報や議論の流れが保持される

* **期待される効果**:
  - 講師の主張とその根拠の関係性を正確に把握できる
  - 概念の定義とその適用範囲を正しく理解できる

この改善により、モデルが講義内容の本質をより正確に理解し、一貫性のある事実に基づいた回答を生成することが期待されます。

In [24]:
# 前後それぞれ2つずつの文章を一つのドキュメントに追加する。（要は5つの文章集合になる)
references = "\n".join(["* " + "。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるChinchilla scaling lawsとは？"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [25]:
print(response)

LLM（Large Language Model）におけるChinchilla scaling lawsは、ChinChilla則という経験則に基づくスケーリング則です。この則は、巨大なモデルにおいて、パラメータ数、token数、計算量の関係を捉えるために用いられます。

ChinChilla則は、Chinchillaというモデル名に由来し、ChinChilla-Optimalという名称で知られています。この則は、巨大なモデルにおいて、パラメータ数を20倍にすると、token数を28.5倍にすると、計算量が変わらないことを示しています。

この則は、LLMの巨大化に際して、パラメータ数、token数、計算量の関係を捉えるために重要な指標となります。ChinChilla則は、実際のLLMの巨大化において、パラメータ数、token数、計算量の関係を捉えるために用いられます。


In [ ]:
# # 評価 (openai apikeyがある場合のみ実行)
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

## 結果 (文脈付きチャンク化によるRAG)

文脈を含むチャンク化により、モデルの回答の方向性に明確な改善が見られました：

### 改善点
* 「推論時の計算をスケールさせる」という概念を据えて回答
* Inference Time Scalingの基本原理についての理解が向上

### 残存する問題点
* 質問と関連性の低い情報（ノイズ）が混入する

### 問題分析

文脈付きチャンク化によるアプローチで新たに発生した課題：

1. **情報過多の問題**:
   * ドキュメント量の増加により、モデルに提供される情報総量が大幅に増加
   * 関連情報と非関連情報が混在し、ノイズと重要情報の区別が困難に

2. **情報選択の複雑化**:
   * モデルは単に回答を生成するだけでなく、提供された多様な情報源から関連性の高い情報を選別する作業も担うことになった
   * この二重タスクにより回答生成の難易度が上昇

# 4. Rerankによる情報品質の向上

検索精度をさらに向上させるため、二段階の検索プロセスを導入します：

* **Rerank手法の導入**:
  - 第一段階: 従来通り基本的な検索アルゴリズムでtop-k個のドキュメントチャンクを取得
  - 第二段階: 取得したチャンクに対してLLMを活用した高度な関連性評価を実施
  - LLMに「このドキュメントは質問『LLMにおけるInference Time Scalingとは？』に対して本当に関連性が高いか」を判断させる
  - 関連性スコアに基づいてランク付けし、真に関連性の高いチャンクのみを選出

* **期待される効果**:
  - 質の高い情報に焦点を絞ることで、ノイズとなる情報を大幅に削減
  - 文脈を保ちながらも、関連性の高い情報のみをモデルに提供
  - モデルのタスクを「多量の情報から選別して回答」から「厳選された情報に基づいて回答」へと単純化

この高度な情報フィルタリングにより、Inference Time Scalingに関する正確で一貫性のある回答生成が期待されます。

上記より、上位3件のみが関連しているとわかったので、これらだけをモデルに渡すこととする。

In [ ]:
# # 評価 (openai apikeyがある場合のみ実行)
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

In [26]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
references = []
for ref in ["。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]]:

  system_prompt = "与えられた参考資料が質問に直接関連しているか？'yes''no'で答えること。ただし、余計なテキストを生成しないこと。"
  question =  f"[参考資料]\n{ref}\n\n[質問] LLMにおけるChinchilla scaling lawsとは？"
  response = generate_output(question, system_prompt)

  print("\n\n対象となるドキュメント:\n", ref.replace("。", "。\n"))
  print("\n関連しているかどうか: ", response)

  if "yes" in response.lower():
    references.append(ref)

  print("\n\n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.




対象となるドキュメント:
 それから1変数を制御するのではなくて複数の変数を制御するような経験則も知られていて、有名なのではChinChillaと呼ばれる経験則があります。
ChinChilla論文って呼ぶ方が正しいかもしれない。
ここまでで計測スケール則の話をしたんですけどChinChillaのところで少しパラメータ数とデータセットサイズ、それぞれいじって計算量を固定しますよって話をしたのでちょっとその補足をしておきたいと思います。
これよく出てくる式、あの経験則の近似式なんですけど、学習に必要な計算量ってどうやって計算してるんですかっていう話があると思います

関連しているかどうか:  yes





The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.




対象となるドキュメント:
 それから逆に右側が横軸がコンピュートになっていて、色がモデルサイズであることは変わらないんですけどこれを見ると例えば10のマイナス3乗の計算量があるときには、これぐらいのコンピュータを使えばいいということがわかったりします。
これ別に大きければいいというわけではないと。
この辺の理屈は小さなモデルだと、学習がロスが下がらなくなるというのでちょっとあのスケール則を書いてるときに、なんか1個1個のプロットを説明したと同じような話ですけど、あの計算量が与えられたときに、どうやら、別にパラメータを増やせばいいわけではないいうことはわかりこの計算量が与えられたときにどれぐらいのバジェット、どれぐらいのパラメータ数とtoken数に割り振ればいいのかっていうのを、あの計算しようとしたのが、先ほど出したこの2変数の関係っていうふうに言ったChinChillaと呼ばれる経験則になってます。
ChinChillaはモデルの名前なんですけどなんかChinChilla則って言われたり、ChinChillaケースって言われたりするので、何かその辺を丸ごと足してChinChillaというふうに、大体呼ばれてると思えばと思います

関連しているかどうか:  yes





The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.




対象となるドキュメント:
 20倍のtokenを利用しましょうということでしたけど、例えばLlama2だと学び7Billonでも70Billonと同じtokenを使ってますけど、7Billonのものに関しては1.8Trillionのtoken使ってるので、285倍の係数と全然違う値を使ってる。
70Billonは28.5倍。
Llama3に関しても、70Billonが214倍、400Billonの方が37倍ということで、このChinChilla-Optimalが実際に巨大なモデル、巨大なtokenの学習で使っているケースもよくあります。
ここまでのまとめが、スケール則ををどう活用するかって話をしてきました

関連しているかどうか:  yes





The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.




対象となるドキュメント:
 黄色が一番大きくて青が一番ちっちゃい、紫かな、ちっちゃいモデルになってます。
先行研究、さっきの今まで話してたものより2桁オーダーが大きいモデルにおいても、これ厳密に言うとまだ収束してないように見えるんですけど、おおむねスケール則が成立してるっていうことがGPT3の論文だと報告されています。
あのスケール則についてここまで他の多分一番有名なのが先ほどのから話しているScaling Laws for Neural Language Modelというものなんですけれど、実はこのスケーリングっていうの自体、スケーリング則、スケール則が成立するってこと自体は、もうちょっと前から知られていたというふうに言われています。
ちょっと僕ももしかしたらもっと昔からあるかもしれないんで僕が知ってる限りですが、少なくとも2017年の論文では検証されているということが言われてます

関連しているかどうか:  no





対象となるドキュメント:
 結果としてはこれで多くのケースより巨大なモデルに勝てるということが実験上示されています。
これ左側のやつが実験上の結果じゃないんで、ちょっと実験結果飲みたい人はこの元の論文を見てもらえれば良いと思いますけど、あの巨大なモデルにかかってるということでこの関係性が良さそうだということが示されています。
ちなみにこれも余談ですけどこのの求め方このChinChilla則ってのは実は何かいくつかの方法で求められてまして、それぞれ大体同じような経験則が出るってことが知られています。
この関係式っていうのがよくこれも知られてまして、大体最適なtoken数っていうのが、パラメータ数に20をかけたもの

関連しているかどうか:  yes





In [27]:
print(len(references))

4


In [28]:
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるChinchilla scaling lawsとは？"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [29]:
print(response)

LLM（Large Language Model）におけるChinchilla scaling lawsは、ChinChillaと呼ばれる経験則に基づくスケール則です。この経験則は、巨大なモデルや巨大なトークンに対する学習の計算量を予測するために使用されます。

この経験則は、ChinChillaの論文に基づいており、パラメータ数とトークン数の関係を示しています。具体的には、パラメータ数を20倍にすると、トークン数も20倍になるという関係を示しています。

この関係式は、LLMの学習に必要な計算量を予測するために使用されます。例えば、巨大なモデルに対する学習には、より多くのトークンが必要になるため、この関係式を使用して、トークン数を計算することができます。

また、この経験則は、LLMの性能を向上させるために使用されます。例えば、パラメータ数を20倍にすると、トークン数も20倍になるため、LLMの性能も向上することができます。


## 結果 (Rerank導入後)

Rerankの導入により、回答品質に改善が見られました：

### 達成された成果
* Inference Time Scalingに関する正確な情報を含んだ回答の生成
* 無関係な情報やノイズの排除
* 講義内容を反映した説明の実現 🎉

この結果から、RAGパイプラインにおける情報の質と関連性の重要性であり、検索で取得した情報を単に増やすだけでなく、その情報の関連性を精査する方法を学ぶことができました。

---


# 5. さらなる改善案: 意味的チャンク化

文単位での分割と前後文脈の追加という現在のアプローチをさらに発展させる手法として、**意味的なチャンク化**が考えられます：

* **意味的チャンク（段落）単位での分割**:
  - 単純な文の区切りではなく、意味的なまとまり（トピック、議論、例示など）に基づいてテキストを分割
  - 人間の主観に基づく意味的な段落分けを活用
  - 各チャンクが「一つの完結した考え」を表現するようにする

* **期待される効果**:
  - より自然な文脈理解が可能に（人間の思考や会話の流れに近い）
  - トピックの開始から結論までの流れを維持できる
  - 概念間の関係性や比較が同一チャンク内に含まれ、より深い理解につなげる

* **検証方法**:
  - 人間が主観的に意味でグループ化したチャンクセットを用意
  - 同じRerank手法を適用し、文単位チャンクとの性能差を比較
  - 回答の正確性、一貫性、網羅性を評価指標として使用

この意味的チャンク化手法は、特に講義のような構造化された発話においては、より自然で効果的な情報検索と理解を可能にすると予想されます。

In [30]:
# 本来は段落をそのままdocumentsに入れずに一定のサイズに分割した方が良いでしょうが、簡単のために段落をそのまま入れてしまいます。
documents = [text.replace("\n", " ").strip() for text in raw_writedown.split("\n\n")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[30])

ドキュメントサイズ:  45
ドキュメントの例: 
 具体的な求め方についても話します。 さっきからチラチラ言ってた通りなんすけど基本的にこれどう図るかっていうと、基本的にはいくつかの条件で実験してフィッティングするって言ってんのは、すごい単純に言ってしまうとそうなります。左側GPT4の論文から取ってきた図で説明したもんですけど、グレーのやつを例えば実験してみて、これぐらいのロスになるんだなっていうので、フィッティングするとこういうカーブになります。 ちなみにこれ、なんでこれ直線にならないんだっていうのをすぐ説明しなかったですがこれ縦軸が実は普通のロスと違ってBits-per-wordっていうのになってて、多分2乗スケールのロスになってるからだと思います。 右側も同じですね。この各点について何かいろんな設定で実験してやって、それを結果を見るということをしてますけどよくよく考えるとスケールさせるときにモデルサイズどうすればいいんでしたっけとか、何をどういじるとモデルサイズが大きくなるんでしたっけ、どういうふうに言えばいいんでしたっけとかですね。 あのモデルサイズ変えたときにハイパーパラメータってどうすんでしたっけそういった細かい問題が出てくる。最初の方ですけどモデルサイズどう変化させるかっていうので、前回やった、こういう図があると思いますけどモデルサイズ変えようと思ったら別にパラメータ、層の数を増やしても、いいわけですし、この埋め込みの次元各tokenの次元を増やしてもいいわけですし、各随所に出てくるこのフィードフォワードネットワークっていうのの中間層の次元を上げてもいいですしヘッドを増やしてもそういうのあのパラメータ自体は上がるということで、これどれをどのぐらいやるんですかっていうのが細かく考えると重要になってきます。 この辺は元の論文でも一応議論されてまして、これ三つほど出してるんすけど例えば真ん中のがアスペクト比っていう、モデルのエンベディングのサイズですね。dモデルっていうものを層数で割ったもの、アスペクト比という縦横比みたいなもので幅と深さの比率をアスペクト比っていうふうにこの論文では呼んでいますけど。こういったものを変えて実験してみたっていうのが最初の最初じゃないOpenAIのScaling Lawで話されていました。基本的にはこの辺見るとなんかあんまり性

In [31]:
question = "LLMにおけるChinchilla scaling lawsとは？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[58.62751007080078, 57.85238265991211, 57.22877883911133, 57.840572357177734, 61.153446197509766, 59.99169158935547, 56.957550048828125, 55.44834899902344, 60.02470779418945, 62.326114654541016, 59.74757766723633, 62.93877410888672, 60.872562408447266, 57.214962005615234, 57.46445083618164, 63.33745574951172, 64.06803131103516, 60.16014862060547, 55.05218505859375, 59.358978271484375, 56.49138259887695, 59.054744720458984, 61.55015182495117, 63.807613372802734, 60.87043380737305, 63.447021484375, 60.91286087036133, 57.454437255859375, 56.79547119140625, 57.59514617919922, 63.183963775634766, 61.365108489990234, 58.448768615722656, 58.96709442138672, 60.12055969238281, 59.38710403442383, 54.35817337036133, 54.19976806640625, 53.446434020996094, 53.69739532470703, 53.49708938598633, 54.25737380981445, 52.76856231689453, 55.05937194824219, 62.04393005371094]]


In [32]:
# 簡単のためにtop2でやります。結果を見てもらえれば問題なく関連する項目のみ取得できているのが分かるかと思います。
topk = 2
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 64.06803131103516)
ここまでがあのスケール則とは何かのまとめでして、スケール則っていうのは毎回おさらいすると、計算資源とデータセット、パラメータと誤差の間にはこういった経験則がそうですよ。 こういうべき乗則で書けますよっていうのがスケール則でした。両対数グラフで線形なるのは両対数をとってやるとわかるということも説明しました。それから一番有名なのはTrasnformerで本当のスケール則ですけど、それ以外のモデルでも成立しますし、言語以外のタスクでもスケール則ってのは確認されていますという話をしました。 それから1変数を制御するのではなくて複数の変数を制御するような経験則も知られていて、有名なのではChinChillaと呼ばれる経験則があります。ChinChilla論文って呼ぶ方が正しいかもしれない。 


取得したドキュメント2: (Score: 63.807613372802734)
ChinChillaはモデルの名前なんですけどなんかChinChilla則って言われたり、ChinChillaケースって言われたりするので、何かその辺を丸ごと足してChinChillaというふうに、大体呼ばれてると思えばと思います。 左側の図は先ほど見せたのと同じで、それぞれの色が計算量に相当してまして、パラメータ数を変更させた場合です。右側の図が増えてるんですけど、これを各FLOPSで最適なパラメータに直したものっていうのが、この中央でこれを同じようにデータ(token数)に対して、直したものが中央になります。 例えばこれパラメータ見ると何か例えば3E-21を使えるんだったら、あのこの一番下のやつをピックアップしたやつか、この右側の真ん中の方に行ってきていて、同じように1E-12の場合はぴって引っ張ってくるみたいな、やったときにどういう関係があるかっていうので、これを見てみると何となく大体線形っぽい関係にわかります。 右側がtokenの場合の同様の例ですね。これをフィッティングしてると、例えばこれ適当な値ですけど10^24よりちょっと低いぐらいの計算量が使えますよっていうときには、パラメータ数は63Billonにすればいいと。同じところが、これ取られてるんすけど、データ数がどれぐらいすればいいかっていうと、1.4Trill

In [35]:
reference = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
question =  f"[参考資料]\n{references}\n\n[質問] LLMにおけるChinchilla scaling lawsとは？"
response = generate_output(question, system_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [36]:
print(response)

LLM（Large Language Model）におけるChinchilla scaling lawsは、ChinChillaと呼ばれる経験則に基づくスケール則です。この経験則は、巨大なモデルや巨大なトークンに対する学習の計算量を予測するために使用されます。

この経験則は、ChinChillaの論文に基づいており、パラメータ数とトークン数の関係を示しています。具体的には、パラメータ数を20倍にすると、トークン数も20倍になるという関係を示しています。

この関係式は、LLMの学習に必要な計算量を予測するために使用されます。例えば、巨大なモデルに対する学習には、より多くのトークンが必要になるため、この関係式を使用して、トークン数を計算することができます。

また、この経験則は、LLMの性能を向上させるために使用されます。例えば、パラメータ数を20倍にすると、トークン数も20倍になるため、LLMの性能も向上することができます。


In [ ]:
# # 評価 (openai apikeyがある場合のみ実行)
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)